# Metric 1: Update Completeness


### Rabbit Hole
* `_extract_ts_local` doesn't always lead up to the stop's actual arrival, or even the max(stop's predicted arrival). If we stop asking, should we penalize? 
* Right now, we'll only count the trip updates for as much as we're asking.
* If `_extract_ts` is not present, we're not asking, then that's a different issue.
* Notice that if we subset to prediction durations, we might lose a lot of rows.

In [1]:
import pandas as pd

import utils
from segment_speed_utils.project_vars import (PREDICTIONS_GCS, 
                                              analysis_date)

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/dask_geopandas/backends.py:13: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/l

In [2]:
def atleast2_updates_by_trip_stop(
    df: pd.DataFrame,
    timestamp_col: str = "_extract_ts_local",
    metric_timestamp_col: str = "trip_update_timestamp_local"
) -> pd.DataFrame: 
    """
    For every trip-stop-minute combination,
    count the number of unique trip_update_timestamps.
    (Checked that this is 3 max).
    If that minute has at least 2, flag that as passing.
    
    Sum up the number that of passing for that stop and 
    calculate the percent. The denominator is the number of 
    trip_min_elapsed.
    
    Note: size here used to count number of rows as denominator.
    But, if we are not asking for predictions (`_extract_ts`), 
    we are also not going to penalize operator for not having predictions
    leading up to the stop.
    """
    all_stop_cols = [
        "gtfs_dataset_key", "_gtfs_dataset_name", 
        "service_date", 
        "shape_id", "route_id",
        "trip_id", 
        "stop_id", "stop_sequence",
        "scheduled_arrival", "actual_stop_arrival_pacific", 
    ]
    minute_cols = [f"{timestamp_col}_hour", f"{timestamp_col}_min"]
    
    # Count for every stop-min, how many unique trip updates
    df2 = (df.groupby(all_stop_cols + minute_cols)
           .agg({metric_timestamp_col: "nunique"})
           .reset_index()
    )
    
    # 1 if it has more than 2 updates, 0 otherwise.
    # Easier to sum and calculate percent.
    df2 = df2.assign(
        atleast2_trip_updates = df2.apply(
            lambda x: 1 if x[metric_timestamp_col] >= 2
            else 0, axis=1)
    )    
    
    # Size: gets us number of rows for that stop
    df3 = (df2.groupby(all_stop_cols)
           .agg({
               f"{timestamp_col}_hour": "size",
               "atleast2_trip_updates": "sum"})
           .reset_index()
          ).rename(columns = {
            f"{timestamp_col}_hour": "trip_min_elapsed"
    })
    
    df3 = df3.assign(
        pct_update_complete = df3.atleast2_trip_updates.divide(
            df3.trip_min_elapsed)
    ) 
    
    return df3

In [3]:
def update_completeness_metric(df: pd.DataFrame) -> pd.DataFrame:
    """
    Start with assembled RT stop_time_updates with 
    scheduled stop_times and also final_trip_updates columns.
    
    For a given stop, if there are predictions/rows present because
    of _extract_ts after the "actual stop arrival" (final_trip_updates), 
    exclude those.
    """
    # Set timestamp columns here, in case these are not correct
    # Row should be derived from _extract_ts (convert to minute combinations)
    # along with stop identifiers
    # For metric, we want to get # unique trip updates
    timestamp_col = "_extract_ts_local"
    metric_col = "trip_update_timestamp_local"
  
    df2 = utils.exclude_predictions_after_actual_stop_arrival(
        df, timestamp_col)
    df3 = utils.parse_hour_min(df2, [timestamp_col])
    
    df4 = atleast2_updates_by_trip_stop(
        df3, 
        timestamp_col,
        metric_col
    )
    
    return df4

In [4]:
df = pd.read_parquet(
    f"{PREDICTIONS_GCS}rt_sched_stop_times_{analysis_date}.parquet", 
)
df._gtfs_dataset_name.unique()

array(['Anaheim Resort TripUpdates',
       'Bay Area 511 Dumbarton Express TripUpdates',
       'Bay Area 511 Fairfield and Suisun Transit TripUpdates'],
      dtype=object)

In [5]:
by_trip_stop = update_completeness_metric(df)

In [6]:
def quick_descriptives(df: pd.DataFrame, 
                       operator: str,
                       cols_to_describe: list):
    print(f"------------- {operator}-------------")
    subset_df = df[df._gtfs_dataset_name==operator] 
    
    for c in cols_to_describe:
        print(subset_df[c].describe())
        print("\n")

In [7]:
cols = [
    "atleast2_trip_updates", 
    "trip_min_elapsed",
    "pct_update_complete"]

for i in by_trip_stop._gtfs_dataset_name.unique():
    quick_descriptives(by_trip_stop, i, cols)


------------- Anaheim Resort TripUpdates-------------
count    265.000000
mean      23.698113
std       18.645197
min        0.000000
25%        7.000000
50%       21.000000
75%       38.000000
max       60.000000
Name: atleast2_trip_updates, dtype: float64


count    265.000000
mean      24.603774
std       18.775528
min        1.000000
25%        8.000000
50%       22.000000
75%       38.000000
max       61.000000
Name: trip_min_elapsed, dtype: float64


count    265.000000
mean       0.927220
std        0.124070
min        0.000000
25%        0.941176
50%        0.962963
75%        0.983607
max        1.000000
Name: pct_update_complete, dtype: float64


------------- Bay Area 511 Dumbarton Express TripUpdates-------------
count    1424.000000
mean       47.509831
std        16.144330
min         6.000000
25%        39.000000
50%        55.000000
75%        60.000000
max        60.000000
Name: atleast2_trip_updates, dtype: float64


count    1424.000000
mean       48.806180
std      